Perbandingan Peringkas Berita Model Pretrained BLOOM dan T5

---

Daniel Nomolas Wicaksono (A11.2022.14448) STKI A11.4701

---

Permasalahan

Dengan banyaknya berita yang beredar, peringkas otomatis menjadi solusi untuk mendapatkan informasi inti dengan cepat. 
Namun, efektivitas model peringkas tergantung pada kemampuan memahami konteks. 
Permasalahan utama adalah menentukan model mana yang menghasilkan ringkasan lebih baik.


---

Ringkasan Proyek
Proyek ini bertujuan untuk membandingkan kemampuan dua model NLP dalam meringkas berita: BLOOM (model pretrained) dan T5 (model yang dilatih ulang). 
Analisis dilakukan untuk mengevaluasi akurasi dan kualitas hasil ringkasan dari kedua model berdasarkan dataset berita.

---

Tujuan proyek :
1. Menerapkan model BLOOM dan T5 untuk meringkas berita.
2. Membandingkan hasil ringkasan kedua model menggunakan metrik tertentu.




In [1]:
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
import torch

from tqdm import tqdm
torch.set_default_tensor_type(torch.cuda.FloatTensor)
device = torch.device('cuda')

c:\Users\danie\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\danie\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\tensor\python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)


Sebelumnya kita import library yang dibutuhkan berikut adalah penjelasan dari librarynya.

pandas: Untuk manipulasi data tabular.
re: Untuk manipulasi teks menggunakan ekspresi reguler.
transformers: Untuk menggunakan tokenizer dan model berbasis BLOOM dari Hugging Face.
torch: Untuk bekerja dengan tensor dan menjalankan model di GPU.

Setelah itu kita mengatur tensor default menggunakan tipe tensor GPU dan memastikan penggunana GPU untuk mempercepat komputasi

In [2]:
def preprocess(text):
    text = re.sub('\s+', ' ', re.sub('\n+', ' ', text.strip()))
    text = re.split(r'(?<=[.!?]) +', text)
    text = text[:5]
    text = ' '.join(text)
    return text

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\danie\AppData\Local\Temp\ipykernel_12644\563678054.py:2: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', re.sub('\n+', ' ', text.strip()))


Selanjutnya kita menentukan proses preprocessing , berikut adalah prosesnya

Menghapus karakter baru (\n) dan mengganti spasi berlebih dengan satu spasi.
Membagi teks menjadi kalimat menggunakan regex (?<=[.!?]) +.
Mengambil maksimal 5 kalimat pertama.
Menggabungkan kembali kalimat-kalimat tersebut menjadi satu teks.

Lalu kita masukkan dataset yang kita dapatkan dari kaggle dengan link  : https://www.kaggle.com/datasets/sh1zuka/indonesia-news-dataset-2024

dataset tersebut memiliki beberapa attribut, yaitu : Judul, Waktu, Link, Content, tag1-5 dan source
Penjelasan attribut :
Judul : Merupakan judul berita.
Waktu : Kapan berita tersebut terbit.
Link :  Link berita tersebut.
Content : Isi text dari berita tersebut.
Tag1-5 : Kata kunci pada berita tersebut.
Source : Asal berita tersebut

In [3]:
df = pd.read_csv('politik_merge.csv')
df.head()

,Judul,Waktu,Link,Content,tag1,tag2,tag3,tag4,tag5,source
0,Jokowi Kenakan Pakaian Adat Betawi di Sidang T...,16/08/2024,https://nasional.kompas.com/read/2024/08/16/11...,"JAKARTA, KOMPAS.com - Presiden Joko Widodo me...",Presiden Jokowi,Jokowi,sidang tahunan MPR RI 2024,Jokowi adat Betawi sidang mpr 2024,Megawati tak hadiri sidang tahunan MPR 2024,kompas
1,Amnesty International Beberkan 6 Indikator Kri...,2024-07-18,https://nasional.tempo.co/read/1893144/amnesty...,"TEMPO.CO, Jakarta - Amnesty International Indo...",Amnesty International,Amnesty International Indonesia,Kebebasan Berpendapat,Indeks Demokrasi,Revisi UU TNI,tempo
2,"Jelang Long Weekend, Stasiun Kereta Cepat Hali...","Rabu, 08 Mei 2024 19:18 WIB",https://news.detik.com/berita/d-7331666/jelang...,"Stasiun kereta cepat Whoosh di Halim, Jakarta ...",kereta cepat whoosh,stasiun halim,long weekend,NaN,NaN,detik
3,KPU Tegaskan Pemilih Tak Terdaftar di DPT Bisa...,13/02/2024,https://nasional.kompas.com/read/2024/02/13/21...,"JAKARTA, KOMPAS.com - Komisi Pemilihan Umum (...",KPU,pemilu 2024,Hasyim Asy'ari,NaN,NaN,kompas
4,Kemenag Luncurkan Gerakan Senam Haji Jaga Keta...,2024-04-29,https://nasional.tempo.co/read/1861810/kemenag...,"TEMPO.CO, Jakarta - Kementerian Agama atau Kem...",Senam Haji,Kemenag,Jemaah Haji,Ibadah Haji,Asrama Haji,tempo


Setelah memasukkan dan mengecek kepala dataset, kita mengapply fungsi preprocessing yang kita buat tadi. Tetapi ketika mengecek sekilas datasetnya terdapat beberapa baris content/ clean yang terisi oleh nilai float yang nantinya akan menganggu proses, jadi kita tambahkan fungsi preprocessing baru disini dengan menghapus baris yang memiliki nilai float.

Lalu kita apply preprocessingnya.

In [5]:

temp_df = df.iloc[:100].copy() #ganti df.iloc ke df.copy bila ingin melakukan pembacaan semua data set
temp_df['Content'] = temp_df['Content'].fillna('')

# Terapkan fungsi preprocess
temp_df['clean'] = temp_df['Content'].apply(preprocess)




temp_df[['Content','clean']]

,Content,clean
0,"JAKARTA, KOMPAS.com - Presiden Joko Widodo me...","JAKARTA, KOMPAS.com - Presiden Joko Widodo mem..."
1,"TEMPO.CO, Jakarta - Amnesty International Indo...","TEMPO.CO, Jakarta - Amnesty International Indo..."
2,"Stasiun kereta cepat Whoosh di Halim, Jakarta ...","Stasiun kereta cepat Whoosh di Halim, Jakarta ..."
3,"JAKARTA, KOMPAS.com - Komisi Pemilihan Umum (...","JAKARTA, KOMPAS.com - Komisi Pemilihan Umum (K..."
4,"TEMPO.CO, Jakarta - Kementerian Agama atau Kem...","TEMPO.CO, Jakarta - Kementerian Agama atau Kem..."
...,...,...
95,"TEMPO.CO, Riau - Kebijakan Merdeka Belajar sa...","TEMPO.CO, Riau - Kebijakan Merdeka Belajar saa..."
96,"MEKKAH, KOMPAS.com- Lukman (bukan nama sebena...","MEKKAH, KOMPAS.com- Lukman (bukan nama sebenar..."
97,"JAKARTA, KOMPAS.com - Ketua Harian Partai Ger...","JAKARTA, KOMPAS.com - Ketua Harian Partai Geri..."
98,Lawrence Wong dilantik sebagai Perdana Menteri...,Lawrence Wong dilantik sebagai Perdana Menteri...


In [6]:

model_name = "bigscience/bloomz-1b1"
model_dir = "saved_model"
try:
    # Load saved model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    model = AutoModelForCausalLM.from_pretrained(model_dir)
    print("Model and tokenizer loaded from disk.")
except:
    # If not saved, download and save them
    print("Downloading model and tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
    model = AutoModelForCausalLM.from_pretrained(model_name, use_cache=True)
    # Save the model and tokenizer
    tokenizer.save_pretrained(model_dir)
    model.save_pretrained(model_dir)
    print("Model and tokenizer saved to disk.")

model.to(device)

Model and tokenizer loaded from disk.


BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1536)
    (word_embeddings_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1536, out_features=4608, bias=True)
          (dense): Linear(in_features=1536, out_features=1536, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1536, out_features=6144, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=6144, out_features=1536, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
  )
  (

In [7]:
prompt = "Mysteriously long text..."

summary_instruction = f"\n\nSummarize the previous text in three sentences:\n\n"

total_prompt = prompt + summary_instruction

In [8]:
xe = '\n\nRingkas teks berita di atas menjadi sebuah kalimat inti.'
print(len(xe))

58


In [9]:
inputs = []
for _, row in temp_df.iterrows():
    txt = row['clean']
    prompt = (f'{txt}\n\nRingkas teks berita di atas menjadi sebuah kalimat inti.')
    input_ids = tokenizer(
                    prompt,
                    return_tensors="pt",
#                     padding="max_length",
#                     truncation=True,
#                     max_length=512
                )["input_ids"].to(device)
    inputs.append(input_ids)

In [10]:
from tqdm import tqdm

In [11]:
outputs = []
for ids in tqdm(inputs):
    output = model.generate(
                input_ids=ids,
                max_length= 500,
                no_repeat_ngram_size=2,
                num_beams=4
            )[0]
    outputs.append(output)

100%|██████████| 100/100 [02:07<00:00,  1.28s/it]


In [12]:
summaries = []
for output in outputs:
    summary = tokenizer.decode(
        output,
        skip_special_tokens=False
    )
    summaries.append(summary)

In [13]:
temp_df['summary'] = summaries
temp_df[['clean', 'summary']]

,clean,summary
0,"JAKARTA, KOMPAS.com - Presiden Joko Widodo mem...","JAKARTA, KOMPAS.com - Presiden Joko Widodo mem..."
1,"TEMPO.CO, Jakarta - Amnesty International Indo...","TEMPO.CO, Jakarta - Amnesty International Indo..."
2,"Stasiun kereta cepat Whoosh di Halim, Jakarta ...","Stasiun kereta cepat Whoosh di Halim, Jakarta ..."
3,"JAKARTA, KOMPAS.com - Komisi Pemilihan Umum (K...","JAKARTA, KOMPAS.com - Komisi Pemilihan Umum (K..."
4,"TEMPO.CO, Jakarta - Kementerian Agama atau Kem...","TEMPO.CO, Jakarta - Kementerian Agama atau Kem..."
...,...,...
95,"TEMPO.CO, Riau - Kebijakan Merdeka Belajar saa...","TEMPO.CO, Riau - Kebijakan Merdeka Belajar saa..."
96,"MEKKAH, KOMPAS.com- Lukman (bukan nama sebenar...","MEKKAH, KOMPAS.com- Lukman (bukan nama sebenar..."
97,"JAKARTA, KOMPAS.com - Ketua Harian Partai Geri...","JAKARTA, KOMPAS.com - Ketua Harian Partai Geri..."
98,Lawrence Wong dilantik sebagai Perdana Menteri...,Lawrence Wong dilantik sebagai Perdana Menteri...


In [14]:
summary_only = []
for _, row in temp_df[['clean', 'summary']].iterrows():
    summ = row['summary'][(len(row['clean'])+58):]
    summ = summ[:-4]
    summary_only.append(summ)
    
temp_df['summary'] = summary_only

In [15]:
for _, row in temp_df[['clean', 'summary']].iterrows():
    print('teks: ', row['clean'], '\n')
    print('sum: ', row['summary'], '\n')

teks:  JAKARTA, KOMPAS.com - Presiden Joko Widodo memakai baju adat Betawi saat hadir di Sidang Tahunan MPR RI dan Sidang Bersama DPR RI dan DPD RI di Kompleks Perlemen, Senayan, Jakarta pada Jumat (16/8/2024). Baju yang dikenakan Presiden yakni pakaian adat Bangsawan Ujung Serong asal Betawi. Pakaian tersebut berwana hitam dan dilengkapi dengan peci hitam. Menurut Deputi II Kepala Staf Kepresidenan (KSP), Abetnego Tarigan, pakaian adat Betawi yang dikenakan Presiden kali ini punya makna terima kasih untuk Kota Jakarta. “Presiden memaknai pakaian adat betawi sebagai simbol terima kasih untuk Kota Jakarta yang sudah resmi menjadi Ibu Kota sejak tahun 1966,” kata Abetnego dalam keterangan resmi KSP, Jumat. 

sum:   Presiden Jokowi kenakan baju khas Betawi untuk menghadiri sidang tahunan MPR dan sidang bersama DPR, DPD, dan DPRD di kompleks parlemen di Jakarta, Kamis (15/8). 

teks:  TEMPO.CO, Jakarta - Amnesty International Indonesia membeberkan enam indikator yang menunjukkan telah terj

In [16]:
temp_df[['clean', 'summary']].to_csv('hasilsummary3.csv')

In [18]:
import csv
import os

def summarize_text(input_text, model, tokenizer, device):
    """Generate a summary for a given text."""
    prompt = (f'{input_text}\n\nRingkas teks berita di atas menjadi sebuah kalimat inti.')
    input_ids = tokenizer(
        prompt,
        return_tensors="pt"
    )["input_ids"].to(device)

    output = model.generate(
        input_ids=input_ids,
        max_length=500,
        no_repeat_ngram_size=2,
        num_beams=4
    )[0]

    summary = tokenizer.decode(output, skip_special_tokens=False)
    summary_cleaned = summary[len(prompt):-4]  # Remove prompt and special tokens
    return summary_cleaned

def user_input_summary():
    """Allow the user to input text and save the summary to a CSV file."""
    # Nama file CSV
    file_name = "outputBLOOM.csv"

    # Cek apakah file sudah ada
    file_exists = os.path.isfile(file_name)

    # Jika file belum ada, buat dengan header
    if not file_exists:
        with open(file_name, mode="w", newline="", encoding="utf-8") as file:
            writer = csv.writer(file)
            writer.writerow(["input", "output"])

    # Ambil input dari user
    user_text = input("Masukkan teks untuk diringkas: ")
    preprocessed_text = preprocess(user_text)
    summary = summarize_text(preprocessed_text, model, tokenizer, device)

    # Simpan input dan output ke file CSV
    with open(file_name, mode="a", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow([user_text, summary])

    print("\nRingkasan : ",summary, "\n berita telah disimpan ke", file_name)

# Panggil fungsi user_input_summary
user_input_summary()



Ringkasan :   Transportasi di ibukota Indonesia, Jakarta, telah mengalami banyak perubahan dalam beberapa dekade terakhir. telah disimpan ke outputBLOOM.csv


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [7]:
data = pd.read_csv('hasilsummary2.csv')

# Pastikan kolom 'clean' dan 'summary' tidak memiliki nilai NaN
data['clean'] = data['clean'].fillna("").astype(str)
data['summary'] = data['summary'].fillna("").astype(str)


if 'clean' not in data.columns or 'summary' not in data.columns:
    raise ValueError("Dataset must have 'clean' and 'summary' columns")

X = data['clean']
y = data['summary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")
model = model.to(device)


In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


# Fungsi untuk melatih dan menyimpan model
def train_and_save_model(X_train, y_train, save_path, epochs=1):
    model.train()
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    for epoch in range(epochs):
        for i, (full_text, summary) in enumerate(zip(X_train, y_train)):
            # Tokenisasi input dan label
            input_ids = tokenizer.encode("summarize: " + full_text, return_tensors="pt", max_length=512, truncation=True).to(device)
            labels = tokenizer.encode(summary, return_tensors="pt", max_length=150, truncation=True).to(device)
            
            # Forward pass
            outputs = model(input_ids=input_ids, labels=labels)
            loss = outputs.loss
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if i % 10 == 0:  # Print loss setiap 10 iterasi
                print(f"Epoch {epoch+1}, Step {i+1}/{len(X_train)}, Loss: {loss.item()}")
    
    # Simpan model yang sudah di-finetune
    torch.save(model.state_dict(), save_path)
    print(f"Model berhasil disimpan ke {save_path}")

    save_path = ""
    train_and_save_model(X_train,y_train, save_path,epochs=1)

In [7]:
# Muat model dari file
def load_model(load_path):
    model.load_state_dict(torch.load(load_path))
    model.to(device)
    model.eval()
    print(f"Model berhasil dimuat dari {load_path}")

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch 1, Step 1/36624, Loss: 4.431310653686523
Epoch 1, Step 11/36624, Loss: 4.781494140625
Epoch 1, Step 21/36624, Loss: 4.257169246673584
Epoch 1, Step 31/36624, Loss: 4.6598052978515625
Epoch 1, Step 41/36624, Loss: 3.119776487350464
Epoch 1, Step 51/36624, Loss: 3.3899765014648438
Epoch 1, Step 61/36624, Loss: 2.8661491870880127
Epoch 1, Step 71/36624, Loss: 2.6086976528167725
Epoch 1, Step 81/36624, Loss: 3.3427493572235107
Epoch 1, Step 91/36624, Loss: 2.5499298572540283
Epoch 1, Step 101/36624, Loss: 3.302206039428711
Epoch 1, Step 111/36624, Loss: 4.012787342071533
Epoch 1, Step 121/36624, Loss: 2.661132574081421
Epoch 1, Step 131/36624, Loss: 3.2674553394317627
Epoch 1, Step 141/36624, Loss: 3.848839044570923
Epoch 1, Step 151/36624, Loss: 2.9227328300476074
Epoch 1, Step 161/36624, Loss: 4.381155967712402
Epoch 1, Step 171/36624, Loss: 2.3515005111694336
Epoch 1, Step 181/36624, Loss: 3.869765043258667
Epoch 1, Step 191/36624, Loss: 2.831148386001587
Epoch 1, Step 201/36624, 

In [25]:
def summarize_text(input_text):
    """Fungsi interaktif untuk melakukan summarization"""
    model.eval()
    with torch.no_grad():
        input_ids = tokenizer.encode("summarize: " + input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
        outputs = model.generate(input_ids=input_ids, max_length=150, num_beams=4, early_stopping=True)
        summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [ ]:
while True:
    user_input = input("Masukkan teks untuk diringkas (atau ketik 'exit' untuk keluar): ")
    if user_input.lower() == 'exit':
        print("Terima kasih telah menggunakan summarizer!")
        break
    result = summarize_text(user_input)
    print("\nRingkasan:\n", result)

NameError: name 'summarize_text' is not defined

In [ ]:
print(summary)


NameError: name 'summary' is not defined